In [ ]:
from nltk.tokenize import TreebankWordTokenizer
from nltk.probability import FreqDist
import tensorflow as tf
import numpy as np
import re
import string

# HABEAS

In [ ]:
with open('iliad_pope.txt', 'r') as f:
    iliad_raw = f.readlines()

In [ ]:
line_regex = re.compile('  [a-zA-Z]')
footnote_regex = re.compile('\(\d+\)')
iliad_text_list = [footnote_regex.sub('',line.strip()) for line in iliad_raw if line_regex.match(line)]
start_ind = iliad_text_list.index("Achilles' wrath, to Greece the direful spring")
iliad_text_list = iliad_text_list[start_ind:-1]
iliad_text = ' '.join(iliad_text_list).translate(str.maketrans('', '', string.punctuation))
iliad_tokens = [token.lower() for token in TreebankWordTokenizer().tokenize(iliad_text)]

# #PHRASING

The use of phrase detection code didn't really pan out - we ended up with valid phrases scoring somewhere in the middle of the distribution. Going forward without using them.

# FEED ME, SEYMOUR

In [1]:
vocabulary = sorted(set(iliad_tokens))
indices = {word : index for (index, word) in enumerate(vocabulary)}
vocab_size = len(vocabulary)

NameError: name 'iliad_tokens' is not defined

In [3]:
WINDOW_SIZE = 5

def generate_skipgrams(tokens, window_size=WINDOW_SIZE):
    for i,token in enumerate(tokens):
        for j in range(i - window_size, i + window_size +1):
            if j not in range(len(tokens)) or j == i : 
                continue
            yield (tokens[i], tokens[j])

def skipgrams_to_onehot(index_dict, skipgram):
    return (index_dict[skipgram[0]], index_dict[skipgram[1]])

# LET'S GET OUR LE CORBUSIER ON

In [ ]:
EMBED_SIZE = 300

X = tf.placeholder(tf.float32, [vocab_size,1])
y = tf.placeholder(tf.float32, [vocab_size,1])

W_embed = tf.Variable(tf.truncated_normal([EMBED_SIZE, vocab_size]))
b_embed = tf.Variable(tf.truncated_normal([EMBED_SIZE,1]))

W_lookup = tf.Variable(tf.truncated_normal([vocab_size, EMBED_SIZE]))
b_lookup = tf.Variable(tf.truncated_normal([vocab_size]))

pope_vec = tf.matmul(W_embed,X) + b_embed
out_vec = tf.nn.softmax(tf.matmul(W_lookup,pope_vec) + b_lookup)